## <font color='green'> 개와 고양이 이미지 분류하기</font>


[ 참고 ] https://codetorial.net/tensorflow/classifying_the_cats_and_dogs.html


위의 참고 사이트에서 코드를 복사하며 실행

학습시간 상당히 소요


### 1. 데이타 준비하기

[ 자료다운 ] https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip

In [ ]:
import os

# 기본 경로
base_dir = 'dataset/cats_and_dogs_filtered'

train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

# 훈련에 사용되는 고양이/개 이미지 경로
train_cats_dir = os.path.join(train_dir, 'cats')
train_dogs_dir = os.path.join(train_dir, 'dogs')
print(train_cats_dir)
print(train_dogs_dir)

# 테스트에 사용되는 고양이/개 이미지 경로
validation_cats_dir = os.path.join(validation_dir, 'cats')
validation_dogs_dir = os.path.join(validation_dir, 'dogs')
print(validation_cats_dir)
print(validation_dogs_dir)

### 2. 데이타 살펴보기

In [ ]:
train_cat_fnames = os.listdir( train_cats_dir )
train_dog_fnames = os.listdir( train_dogs_dir )

print(train_cat_fnames[:5])
print(train_dog_fnames[:5])

print('Total training cat images :', len(os.listdir(train_cats_dir)))
print('Total training dog images :', len(os.listdir(train_dogs_dir)))

print('Total validation cat images :', len(os.listdir(validation_cats_dir)))
print('Total validation dog images :', len(os.listdir(validation_dogs_dir)))

### 3. 이미지 확인하기

In [ ]:
%matplotlib inline

import matplotlib.image as mpimg
import matplotlib.pyplot as plt

nrows, ncols = 4, 4
pic_index = 0

fig = plt.gcf()
fig.set_size_inches(ncols*3, nrows*3)

pic_index+=8

next_cat_pix = [os.path.join(train_cats_dir, fname)
                for fname in train_cat_fnames[ pic_index-8:pic_index]]

next_dog_pix = [os.path.join(train_dogs_dir, fname)
                for fname in train_dog_fnames[ pic_index-8:pic_index]]

for i, img_path in enumerate(next_cat_pix+next_dog_pix):
    sp = plt.subplot(nrows, ncols, i + 1)
    sp.axis('Off')

    img = mpimg.imread(img_path)
    plt.imshow(img)

plt.show()

### 4. 모델 구성하기

In [ ]:
import tensorflow as tf


model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(150, 150, 3)),
  tf.keras.layers.MaxPooling2D(2,2),
  tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2,2),
  tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2,2),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(512, activation='relu'),
  tf.keras.layers.Dense(1, activation='sigmoid')
])

model.summary()

### 5. 모델 컴파일하기

In [ ]:
from tensorflow.keras.optimizers import RMSprop

model.compile(optimizer=RMSprop(lr=0.001),
            loss='binary_crossentropy',
            metrics = ['accuracy'])

### 6. 이미지 데이터 전처리하기

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator


train_datagen = ImageDataGenerator( rescale = 1.0/255. )
test_datagen  = ImageDataGenerator( rescale = 1.0/255. )

train_generator = train_datagen.flow_from_directory(train_dir,
                                                  batch_size=20,
                                                  class_mode='binary',
                                                  target_size=(150, 150))
validation_generator =  test_datagen.flow_from_directory(validation_dir,
                                                       batch_size=20,
                                                       class_mode  = 'binary',
                                                       target_size = (150, 150))

### 7. 모델 훈련하기


( 에포크가 100이라 시간소요 상당함 )

In [ ]:
'''
history = model.fit(train_generator,
                    validation_data=validation_generator,
                    steps_per_epoch=100,
                    epochs=100,
                    validation_steps=50,
                    verbose=2)
'''

<font color='red'>

* 학습속도가 너무 느려서 모델을 저장하고 다시 불러다 사용

    저장하기 : model.save('./savedata/catndog.h5')

    불어오기 : h5_model=tf.keras.models.load_model('./savedata/catndog.h5')
    
</font>    

In [ ]:
# model.save('./savedata/catndog.h5')

In [ ]:
h5_model=tf.keras.models.load_model('./savedata/catndog.h5')

### 8. 정확도와 손실 확인하기

훈련을 한 후에 그 결과로 history를 이용하는 것이라, 이미 저장된 h5 파일을 이용하면 아래 코드 사용 못함

In [ ]:
'''
import matplotlib.pyplot as plt

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'go', label='Training Loss')
plt.plot(epochs, val_loss, 'g', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()
'''

### 9. 테스트 이미지 분류하기

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image

# path='./dataset/cats_and_dogs_filtered/train/dogs/dog.4.jpg'
# path='./dataset/cats_and_dogs_filtered/train/cats/cat.4.jpg'
path='./imgs/cat.png'
img=image.load_img(path, target_size=(150, 150))

x=image.img_to_array(img)
x=np.expand_dims(x, axis=0)
images = np.vstack([x])

# classes = model.predict(images, batch_size=10)
# 저장된 h5 파일에서 읽은 모델 사용
classes = h5_model.predict(images, batch_size=10)

print(classes[0])

if classes[0]>0:
    print("The picture is a dog")
else:
    print("The picture  is a cat")